# Estimació dels paràmetres del model probabilístic p* i q* i sampling

## Context i motivació

Al notebook anterior (`crear_blueprint.ipynb`) es va construir l’estructura base del problema d’alineació de xarxes, definint el wiring (L_wiring), la xarxa observada (A_test) i la funció d’energia (Hamiltoniana) extretes del repositori del paper, així com diferents procediments de permutació per generar configuracions de referència construïdes reordenant l'observació, permutant etiquetes i fent shuffles; en comptes de mostrejar-les a partir d'un model generatiu.

L’objectiu d’aquest notebook és reformular l’anàlisi des d’un punt de vista probabilístic. En lloc de generar configuracions mitjançant shuffles, es treballa amb un ensemble de xarxes generades directament des del model, mantenint fixa l’estructura del wiring (un cop introduïda la probabilitat) i estimant els paràmetres del model a partir de la xarxa observada.

## Objectius:

- Generar una matriu de wiring (L_wiring) aleatòria, assignant la mateixa probabilitat a tots els enllaços possibles.


- Estimar els paràmetres òptims del model p* iq* a partir de (L_wiring) i de la xarxa observada (A_test).

- Generar un ensemble de xarxes utilitzant els paràmetres estimats i la permutació identitat.

- Calcular la distribució d’energies del model i comparar-la amb l’energia observada per tal d’avaluar la significança estructural de l’alineació.

Aquest enfocament permet construir un model nul coherent amb l’estructura del sistema i analitzar quantitativament fins a quin punt la configuració observada és compatible amb el model probabilístic considerat.

## Generació del wiring aleatori

## Estimació dels paràmetres p* i q*

##  Generació de l’ensemble de xarxes

## Càlcul de l’energia

## Histograma i comparació amb l’observació

## Conclusions